<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Aleja\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
path = 'C:/Users/Aleja/Documents/Universidad/NLP/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto 

data['Pre-Procesado'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [4]:
# Crear una represantación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Procesado'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['Pre-Procesado'].values]

# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 10, passes=10)

###  `[25 pts]` Punto 3: Visualización de LDA

In [5]:
model.print_topics(num_words=15)

[(0,
  '0.035*"regalo" + 0.031*"bonito" + 0.026*"si" + 0.022*"juego" + 0.021*"buen" + 0.018*"colgante" + 0.017*"tamaño" + 0.016*"grande" + 0.015*"pequeño" + 0.014*"esperaba" + 0.014*"roto" + 0.014*"queda" + 0.013*"así" + 0.013*"saga" + 0.012*"parte"'),
 (1,
  '0.035*"llegado" + 0.022*"así" + 0.021*"juego" + 0.021*"si" + 0.020*"pena" + 0.019*"día" + 0.018*"amazon" + 0.017*"regalo" + 0.015*"comprar" + 0.014*"vale" + 0.013*"bien" + 0.011*"llegar" + 0.011*"pedido" + 0.010*"caja" + 0.010*"hijo"'),
 (2,
  '0.026*"juego" + 0.021*"parece" + 0.021*"calidad" + 0.017*"producto" + 0.017*"viene" + 0.014*"tan" + 0.014*"collar" + 0.014*"xbox" + 0.014*"precio" + 0.012*"puede" + 0.012*"bastante" + 0.010*"verdad" + 0.010*"nuevo" + 0.010*"historia" + 0.010*"ser"'),
 (3,
  '0.053*"producto" + 0.039*"bien" + 0.023*"puede" + 0.023*"tiempo" + 0.023*"buen" + 0.022*"solo" + 0.022*"llego" + 0.017*"si" + 0.017*"precio" + 0.016*"después" + 0.016*"pedir" + 0.015*"va" + 0.015*"juegos" + 0.015*"cadena" + 0.013*"cali

In [6]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

In [8]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [10]:
for t in range(0,10):
    top_name = f"topic_{t}"
    data[top_name] = data['Pre-Procesado'].apply(lambda doc: get_doc_top_n(doc, t))

In [15]:
for t in range(0,10):
    print(f"************ TOPIC {t} ************")
    topic = f"topic_{t}"
    for i,articulo in enumerate(data.sort_values(topic, ascending=False)['review_body'].values[:5]):
        print(f"review_body #{i}")
        print(articulo[:500])
        print()
    print()

************ TOPIC 0 ************
review_body #0
Buenas, quiero poner una queja del repartidor que ha traido este pedido, ha entregado el pedido pero a mi no me lo ha dado, ni he recibido ninguna llamada, simplemente ha dado un pedido de 64 euros a un completo desconocido, ruego expulsen de su empresa a ese impresentable, ya que es un pedido para uso personal y ha sido entregado a una direccion en la que ahora no hay viviendo nadie por equivocacion mia, se que he fallado pero lo mas normal es que me llamara o no entregara el pedido... Espero q

review_body #1
Conectado al PC un muy buen mando, mi problema ha sido qué por el motivo que sea no me conecta a la Switch, que es para lo que lo quería, no se si es culpa del mando o culpa mia que no he sabido hacerlo pero no ha habido manera, aún así si hubiera sincronizado tenia pinta de muy buen mando.

review_body #2
Réplica del controlador de la NeoGeo CD para ser usado en NeoGeo Mini. Es una muy buena réplica pero ni la palanca ni la boton

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

Se puede observar que los grupos de este modelo LDA son muy similares a los obtenidos en K-Means, los clasifica en varios temas como quejas del producto y la entrega, relación precio calidad del producto entre otros, también se observa que hay dificultad para definir los grupos como es el caso del **9 y 5** como se muestra en la gráfica. Sin embargo, según mi perspectiva siento que este modelo LDA clasifica de una mejor manera los documentos con respecto a los otros modelos estudiados.
